In [2]:
%run data_read.py

Data path: C:\Users\Pop\Documents\GitHub\utcc_independent_study\training\..\data\300_data_pop.xlsx
data_size: 318
variable: data train_data test_data


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data import Field, TabularDataset, BucketIterator
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import uniform
import pandas as pd

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['comment'], data['score_norm'], test_size=0.2, random_state=42)

# Define the Field objects for preprocessing the text and score columns
text = Field(tokenize=thai_tokenizer)
score = Field(sequential=False, use_vocab=False)

# Load the TabularDataset from the DataFrame
datafields = [('comment', text), ('score', score)]
train, valid, test = TabularDataset.splits(path='.', train='train.csv', validation='valid.csv', test='test.csv', format='csv', fields=datafields, skip_header=True)

# Build the vocabulary for the text Field
text.build_vocab(train)

# Define the device to use for training (CPU)
device = torch.device('cpu')

# Define the model architecture
class Regressor(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        pooled = nn.functional.avg_pool2d(embedded.permute(1, 2, 0), (embedded.shape[1], 1)).squeeze(1)
        out = nn.functional.relu(self.fc1(pooled))
        out = nn.functional.relu(self.fc2(out))
        out = self.fc3(out)
        return out

# Define the hyperparameters for training the model
batch_size = 64
embedding_dim = 100
hidden_dim = 128
output_dim = 1
dropout = 0.5
learning_rate = 0.001
n_epochs = 10

# Create the iterators for training and validation sets
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train, valid, test), batch_size=batch_size, device=device)

# Initialize the model and move it to the device
model = Regressor(len(text.vocab), embedding_dim, hidden_dim, output_dim, dropout).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(n_epochs):
    train_loss = 0
    valid_loss = 0
    model.train()
    for batch in train_iterator:
        text_batch = batch.comment
        score_batch = batch.score_norm.float()
        optimizer.zero_grad()
        output = model(text_batch).squeeze(1)
        loss = criterion(output, score_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_iterator)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}')


OSError: [WinError 127] The specified procedure could not be found